# Les Box

---

Dans ce chapitre, je présenterai rapidement l'allocation dynamique ainsi que différentes structures qui l'utilisent. Contrairement à certains langages haut niveau tel JavaScript, la communauté Rust a pris soin à ce qu'aucune allocation dynamique de mémoire ne soit faite derrière le dos des programmeurs, ainsi **aucune des structures présentes dans ce chapitre n'implémentent le trait Copy**, car une copie implicite de la part du compilateur serait une allocation mémoire et pourrait nuire aux performances du programme.

Cependant, tous ces types implémentent **le trait Clone** et peuvent être donc EXPLICITEMENT copiées via la méthode clone().

In [13]:
let s1 = "Bananes".to_string(); // Creation d'une chaine de caractere allouee dynamiquement
// let s2 = s1;                 // Ceci n'est pas possible si je veux conserver un acces a s1

let s2 = s1.clone();            // Copie explicite donc allocation et creation d'une nouvelle chaine s2
dbg!(s1);
dbg!(s2);

[src/lib.rs:150] s1 = "Bananes"
[src/lib.rs:151] s2 = "Bananes"


> La mémoire occupée par les types alloués dynamiquement est automatiquement libérée dès que l'on perd l'ownership des variables. Il n'y a pas de fuite mémoire possible (sauf code unsafe).


## Introduction

> Une Box fait partie de la famille des pointeurs intelligents, c'est la structure allouée sur la heap la plus simple en Rust :
```
pub struct Box<T, A = Global>(_, _) 
where
    A: Allocator,
    T: ?Sized;
```
- À la différence de malloc en C qui prend une taille en octets, on donne a **Box::new()** le type de donnée qui sera allouée.

In [12]:
#[derive(Debug)]
struct Vector {
    i: i32,
    j: i32,
}

let vec = Box::new(Vector{ i: 10, j: -5});
dbg!(vec);

[src/lib.rs:147] vec = Vector {
    i: 10,
    j: -5,
}


> Ici, une structure de type Vector sera allouée sur le tas et non plus sur la pile.

- Il n'y a pas de garbage collector en Rust, ni besoin de libérer manuellement la mémoire. À la fin du bloc dans lequel elle est déclarée, la Box sera automatiquement détruite et la mémoire prudemment allouée, rendue au système. Si l'on veut faire vivre plus longtemps la Box, il est nécessaire qu'une fonction la retourne :

In [11]:
#[derive(Debug)]
struct MyBox {
    b: Box<u32>,
}
impl Drop for MyBox {
    fn drop(&mut self) {
        println!("The box will be destroyed now !");
    }
}

fn make_my_box(inner: u32) -> MyBox {
    MyBox {
        b: Box::new(inner),
    }
}

fn take_my_box(_my_box: MyBox) {
    // La box ne vivra plus a la fin du cette fonction
}

let b = make_my_box(42);
dbg!(&b);
take_my_box(b); // Transfert l'ownership de la structure contenant la box a la fonction take_my_box
 // A ce point du code, la box a deja ete liberee
println!("END ?");

[src/lib.rs:148] &b = MyBox {
    b: 42,
}


The box will be destroyed now !
END ?


In [10]:
#[derive(Debug)]
struct Vector {
    i: isize,
    j: isize,
}
let mut v = Vector{i: 9, j: -3};
v.i = 87;
v.j = -12;
let mut b = Box::new(Vector{i: 10, j: -20});
b.i = 12;
b.j = -11;
dbg!(v.i);

[src/lib.rs:131] v.i = 87


## le trait Deref

- La Box implémente le trait **Deref** qui permet d'accéder au type contenu. Dans l'exemple ci-dessous, il n'est pas possible d'additionner a et b à cause d'une incompatibilité de type :

In [9]:
let mut my_box: Box<usize> = Box::new(12);
let a: usize = 13;
let cmp = my_box == a;
dbg!(cmp);

Error: mismatched types

- Ce problème se résout en dereferencant la Box : (Grace a la petite étoile)

In [8]:
let my_box: Box<usize> = Box::new(12);
let a: usize = 13;
let cmp = *my_box == a; // Dereferencement de my_box
dbg!(cmp);

#[derive(Eq, PartialEq)]
struct S { i: usize, j: usize}
let my_box = Box::new(S{ i: 4, j: 4});
let b = S{ i: 4, j: 4};
let cmp = *my_box == b; // Dereferencement de my_box
dbg!(cmp);

[src/lib.rs:121] cmp = false
[src/lib.rs:126] cmp = true


- Notez que dans les cas d'accès a un membre d'une structure ou d'un tuple, le dereferencement est automatique et ne nécessite pas la présence de l'opérateur associe :

In [7]:
#[derive(Eq, PartialEq)]
struct S { i: usize, j: usize}
let my_box = Box::new(S{ i: 4, j: 4});
let b = S{ i: 4, j: 4};
let cmp = (*my_box).i == b.i; // Dereferencement de my_box
dbg!(cmp);
let cmp = my_box.i == b.i; // Strictement equivalent (*my_box.i) == my_box.i == usize
dbg!(cmp);

[src/lib.rs:115] cmp = true
[src/lib.rs:117] cmp = true


## Cas d'utilisation d'une Box

> Souvent, l'on utilise une Box pour stocker des objets trop volumineux, ou que l'on ne veut pas placer sur la pile pour certaines raisons, parfois bonnes, parfois moins bonnes. Mais il existe au moins deux cas de figure dans lesquels leur utilisation est nécessaire, l'allocation d'un type de taille indéfini et les traits dynamiques (Bon c'est aussi à cause de la taille indéfinie là).

### Taille inconnue a la compilation

- Par exemple, Le compilateur ne peut connaitre à l'avance la taille de la structure a cause de la slice qui constitue un de ses champs :

In [6]:
struct UndefinedSize {
    a: [u8],
    b: usize,
}

Error: the size for values of type `[u8]` cannot be known at compilation time

- Utiliser une Box règle le problème. Ici, la taille de la structure sera de 24 octets, la Box en prend 16 sur un système 64bits (8 pour l'adresse mémoire + 8 pour la taille de l'objet (respectivement 8 et 16)) :

In [5]:
struct UndefinedSize {
    a: Box<[u8]>,
    b: usize,
}
let u1 = UndefinedSize{
    a: Box::new([0; 8]),
    b: 42,
};
let u2 = UndefinedSize{
    a: Box::new([255; 16]),
    b: 42,
};
dbg!(std::mem::size_of::<UndefinedSize>());
dbg!(u1.a.len());
dbg!(u2.a.len());

[src/lib.rs:114] std::mem::size_of::<UndefinedSize>() = 24
[src/lib.rs:115] u1.a.len() = 8
[src/lib.rs:116] u2.a.len() = 16


### Trait dynamique

Comme on l'a vu tout à l'heure, L'utilisation d'une Box peut être utile lorsque le compilateur ignore la taille du type de retour, c'est ce qui arrive quand on veut que le retour d'une fonction soit un type qui implémente un trait particulier plutôt qu'un type donne. On appelle aussi ce pattern le "dynamic dispatch" ou "trait object".

- L'exemple donné par RustByExemple illustre bien l'utilisation de traits dynamiques avec Box :

In [14]:
{
    struct Sheep {}
    struct Cow {}

    trait Animal {
        // Instance method signature
        fn noise(&self) -> &'static str;
    }

    // Implement the `Animal` trait for `Sheep`.
    impl Animal for Sheep {
        fn noise(&self) -> &'static str {
            "beeeeeeh!"
        }
    }

    // Implement the `Animal` trait for `Cow`.
    impl Animal for Cow {
        fn noise(&self) -> &'static str {
            "meuuuuuh!"
        }
    }

    // Returns some struct that implements Animal, but we don't know which one at compile time.
    fn random_animal(random_number: f64) -> Box<dyn Animal> {
        if random_number < 0.5 {
            Box::new(Sheep {})
        } else {
            Box::new(Cow {})
        }
    }

    fn main(random_number: f64) {
        let animal = random_animal(random_number);
        println!("You've randomly chosen an animal, and it says {}", animal.noise());
    }
    main(0.334);
    main(0.9);
}

You've randomly chosen an animal, and it says beeeeeeh!
You've randomly chosen an animal, and it says meuuuuuh!


()

#### Exercice 

- Proposez une solution alternative au trait dynamique tout en conservant le debut et la fin du code :

In [3]:
{
    struct Sheep {}
    struct Cow {}

    trait Animal {
        // Instance method signature
        fn noise(&self) -> &'static str;
    }

    // Implement the `Animal` trait for `Sheep`.
    impl Animal for Sheep {
        fn noise(&self) -> &'static str {
            "beeeeeeh!"
        }
    }

    // Implement the `Animal` trait for `Cow`.
    impl Animal for Cow {
        fn noise(&self) -> &'static str {
            "meuuuuuh!"
        }
    }

    // Ajoutez ce qui est necessaire ici.

    fn main(random_number: f64) {
        let animal = random_animal(random_number);
        println!("You've randomly chosen an animal, and it says {}", animal.noise());
    }
    main(0.334);
    main(0.9);
}

Error: cannot find function `random_animal` in this scope

> Il est possible de creer une nouvelle Enumeration qui contient les deux animaux au choix.

    Cet exemple d'implementation, bien qu'ajoutant beaucoup au compteur du nombre de lignes de code, fonctionne :




```
{
    struct Sheep {}
    struct Cow {}

    trait Animal {
        // Instance method signature
        fn noise(&self) -> &'static str;
    }

    // Implement the `Animal` trait for `Sheep`.
    impl Animal for Sheep {
        fn noise(&self) -> &'static str {
            "beeeeeeh!"
        }
    }

    // Implement the `Animal` trait for `Cow`.
    impl Animal for Cow {
        fn noise(&self) -> &'static str {
            "meuuuuuh!"
        }
    }

    enum AnimalType {
        Sheep(Sheep),
        Cow(Cow),
    }

    impl Animal for AnimalType {
        fn noise(&self) ->  &'static str {
            match self {
                AnimalType::Sheep(sheep) => sheep.noise(),
                AnimalType::Cow(cow) => cow.noise(),
            }
        }
    }

    fn random_animal(random_number: f64) -> AnimalType {
        if random_number < 0.5 {
            AnimalType::Sheep(Sheep {})
        } else {
            AnimalType::Cow(Cow {})
        }
    }

    fn main(random_number: f64) {
        let animal = random_animal(random_number);
        println!("You've randomly chosen an animal, and it says {}", animal.noise());
    }
    main(0.334);
    main(0.9);
}
```